<a href="https://colab.research.google.com/github/16AbhinavReddy/16AbhinavReddy/blob/main/ANN_Hyperparameter_Tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
data = pd.read_csv("/content/Churn_Modelling.csv")
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [ ]:
data = data.drop(["RowNumber","CustomerId","Surname"],axis=1)

In [ ]:
x = data.drop(["Exited"], axis=1)
y = data["Exited"]

In [ ]:
geo = pd.get_dummies(x["Geography"])
gen = pd.get_dummies(x["Gender"])

In [ ]:
x = pd.concat([x,geo,gen], axis=1)

In [ ]:
x = x.drop(["Geography","Gender"], axis=1)

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.2, random_state=101)

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.fit_transform(x_test)

In [ ]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

In [ ]:
from keras.models import Sequential
from keras.layers import Dense,Activation, Embedding, LeakyReLU, Flatten, BatchNormalization, Dropout
from keras.activations import relu, sigmoid

In [ ]:
def model_create(layers, activations):
  model = Sequential()
  for i, nodes in enumerate(layers):
    if i == 0:
      model.add(Dense(nodes,input_dim=x_train.shape[1]))
      model.add(Activation(activations))
      model.add(Dropout(0.3))
    else:
      model.add(Dense(nodes))
      model.add(Activation(activations))
      model.add(Dropout(0.3))
  model.add(Dense(units=1, kernel_initializer = 'glorot_uniform', activation='sigmoid'))
  model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
  return model

In [ ]:
model = KerasClassifier(build_fn= model_create, verbose=0)

<ipython-input-35-77c9326bedee>:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn= model_create, verbose=0)


In [ ]:
layers = [[20], [40,20], [45,30,15]]
activations = ['relu', 'sigmoid']
param_grid = dict(layers=layers, activations=activations, batch_size=[128,256], epochs=[30])
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=5)

In [ ]:
grid_result = grid.fit(x_train, y_train)

In [ ]:
print(grid_result.best_score_, grid_result.best_params_)

0.8543749928474427 {'activations': 'relu', 'batch_size': 128, 'epochs': 30, 'layers': [40, 20]}


In [ ]:
y_pred = grid.predict(x_test)

63/63 [==============================] - 0s 3ms/step


In [ ]:
y_pred = (y_pred > 0.5)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred)

In [ ]:
accuracy

0.8585